# 1 下载观测和导航数据

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


从IGS官网下载完整测站信息list



In [42]:
import requests

# 目标URL
url = "https://network.igs.org/api/public/stations/?current=true&draw=1&format=json&length=767&ordering=name&start=0"

# 发送GET请求
response = requests.get(url)

# 检查请求是否成功
if response.status_code == 200:
    # 将响应的JSON内容保存到文件
    with open("stations_data.json", "w", encoding="utf-8") as json_file:
        json_file.write(response.text)
    print("文件已成功下载并保存为stations_data.json")
else:
    print(f"请求失败，状态码: {response.status_code}")


文件已成功下载并保存为stations_data.json


输入测站名称，自动获取经纬高信息

In [43]:
import json

#========================变量修改==============================
station_name = 'SAVO'  #《======================================================================================================= 1
#==============================================================

#========================函数定义==============================
# 定义函数，根据name查找station的所有信息
def find_station_fullname_by_name(name):
    for station in stations_data:
        if station['name'][:4] == name:  # 确保station是字典，并匹配name
            return station['name']
    return None

# 定义函数，根据name查找llh信息
def find_llh_by_name(name):
    for station in stations_data:
        if station['name'] == name:  # 确保station是字典
            return station['llh']
    return None
#==============================================================

# 假设你已经正确读取了JSON文件
with open('/content/stations_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

stations_data = data.get('data', [])  # 根据实际键名替换 'stations'
station_full_name = find_station_fullname_by_name(station_name)  # 查找测站的完整名称
llh_info = find_llh_by_name(station_full_name)  # 查找对应完整名称的经纬高信息

if station_full_name:
    print(f"{station_name}的完整名称是: {station_full_name}")
else:
    print(f"未找到名称为 {station_name} 的站点。")

if llh_info:
    print(f"{station_name}的llh信息是: {llh_info}")
else:
    print(f"未找到名称为 {station_name} 的站点。")

SAVO的完整名称是: SAVO00BRA
SAVO的llh信息是: [-12.939246833333332, -38.43225372222221, 76.3]


完善其他变量的定义

In [44]:
import datetime
#####################################下载数据时的信息########################################
start_date = datetime.datetime(2020, 1, 1)  #《======================================================================================================= 2
end_date = datetime.datetime(2021, 12, 31)  #《======================================================================================================= 3
obs_suffix = station_full_name[6:] + '_R'  # 自动获取下载文件的尾缀
#######################################

#############################################处理数据时的信息#############################################
SS_variables = ['S2W']  # ['S1C', 'S1W', 'S2W']  #《======================================================================================================= 4
station_llh = llh_info
el_mask = 20  # 高度截止角
base_data_path = '/content/data' # 数据存储路径
start_year = start_date.year
end_year = end_date.year

####################################生成数据集时的信息####################################
# 定义起始年份和结束年份
signal = SS_variables[0]
# 定义卫星号的范围
satellites = [f'G{str(i).zfill(2)}' for i in range(1, 33)]  # 生成G01到G32的卫星号

#################################SFPD定义的信息####################################
# 可修改变量命名区
#years = ['2020','2021','2022']
years = ['2020'，'2021']
#stations = ['AIRA', 'BAIE', 'BIK0', 'CAS1']
stations = ['HLFX']
svn_range = range(1, 33)  # G01到G32的范围

该部分下载耗时大概 5min/1year

In [45]:

import pandas as pd
from ftplib import FTP_TLS, error_perm, error_temp
import gzip
import shutil
import os
import time

# 安装必要的库（如果还没安装）
!pip install pandas

# 函数：将日期转换为 GPS 周和周内的天数
def date_to_gps_week_day(date):
    gps_start = datetime.datetime(1980, 1, 6)
    delta = date - gps_start
    gps_week = delta.days // 7
    gps_day = delta.days % 7
    return gps_week, gps_day

# 函数：将日期转换为年份和DOY
def date_to_year_doy(date):
    year = date.year
    doy = (date - datetime.datetime(year, 1, 1)).days + 1
    return year, doy

# 函数：连接FTP服务器
def connect_to_ftp():
    print("正在初始化与FTP服务器的连接...")
    try:
        ftp = FTP_TLS('gdc.cddis.eosdis.nasa.gov')
        print("成功建立与FTP服务器的连接。")

        print("正在尝试登录...")
        ftp.login()
        print("登录成功。")

        print("正在切换到安全数据连接...")
        ftp.prot_p()  # 切换到加密的数据连接
        print("已建立安全数据连接。")

        return ftp
    except Exception as e:
        print(f"连接FTP服务器时出错: {e}")
        raise

# 函数：从FTP服务器下载文件并解压
def download_and_extract_files(start_date, end_date, save_path_obs, save_path_sp3, station_name, obs_suffix, download_obs=False, download_sp3=True):
    if not os.path.exists(save_path_obs):
        os.makedirs(save_path_obs)
        print(f"创建目录: {save_path_obs}")
    if not os.path.exists(save_path_sp3):
        os.makedirs(save_path_sp3)
        print(f"创建目录: {save_path_sp3}")

    # 连接FTP服务器
    print("正在连接FTP服务器...")
    ftp = connect_to_ftp()
    print("FTP连接成功建立。")

    day_count = 0
    start_time = time.time()

    current_year = start_date.year

    # 遍历日期范围
    for single_date in pd.date_range(start=start_date, end=end_date):
        print(f"\n正在处理日期: {single_date.strftime('%Y-%m-%d')}")

        if single_date.year != current_year:
            current_year = single_date.year
            save_path_obs = f"/content/data/obs/{station_name}/{current_year}"
            save_path_sp3 = f'/content/data/sp3/{current_year}'
            if not os.path.exists(save_path_obs):
                os.makedirs(save_path_obs)
                print(f"为obs数据创建新目录: {save_path_obs}")
            if not os.path.exists(save_path_sp3):
                os.makedirs(save_path_sp3)
                print(f"为sp3数据创建新目录: {save_path_sp3}")

        gps_week, gps_day = date_to_gps_week_day(single_date)
        year, doy = date_to_year_doy(single_date)
        year_short = single_date.strftime('%y')
        doy_padded = f'{doy:03d}'

        # 下载obs文件
        if download_obs:
            obs_suffix_orig = obs_suffix  # 保存原始的obs_suffix
            obs_filename = f'{station_name}00{obs_suffix}_{year}{doy_padded}0000_01D_30S_MO.crx.gz'
            obs_directory = f'/pub/gps/data/daily/{year}/{doy_padded}/{year_short}d/'
            local_obs_gz_filename = os.path.join(save_path_obs, obs_filename)
            local_obs_filename = os.path.join(save_path_obs, obs_filename[:-3])

            # 检查文件是否已经存在
            if os.path.exists(local_obs_filename.replace(".crx", ".crx")):
                print(f'{local_obs_filename.replace(".crx", ".crx")} 已存在，跳过下载。')
            else:
                max_retries = 2
                retry_delay = 2
                attempts = 0
                file_downloaded = False  # 标志变量，用于跟踪文件是否成功下载
                try_suffixes = [obs_suffix, obs_suffix[:-1] + 'S', obs_suffix[:-1] + 'R']  # 依次尝试的suffix

                for suffix in try_suffixes:
                    obs_suffix = suffix  # 更新obs_suffix
                    obs_filename = f'{station_name}00{obs_suffix}_{year}{doy_padded}0000_01D_30S_MO.crx.gz'
                    local_obs_gz_filename = os.path.join(save_path_obs, obs_filename)
                    local_obs_filename = os.path.join(save_path_obs, obs_filename[:-3])

                    for attempt in range(max_retries):
                        try:
                            print(f"正在下载 {obs_filename}...")
                            with open(local_obs_gz_filename, 'wb') as file:
                                ftp.retrbinary(f'RETR {obs_directory}{obs_filename}', file.write)
                            print(f'成功下载 {local_obs_gz_filename}')

                            print(f"正在解压 {local_obs_gz_filename}...")
                            with gzip.open(local_obs_gz_filename, 'rb') as f_in:
                                with open(local_obs_filename, 'wb') as f_out:
                                    shutil.copyfileobj(f_in, f_out)
                            print(f'成功解压到 {local_obs_filename}')

                            # 删除压缩文件
                            os.remove(local_obs_gz_filename)
                            print(f"删除压缩文件 {local_obs_gz_filename}")
                            file_downloaded = True  # 文件已成功下载
                            break
                        except error_perm as e:
                            if str(e).startswith('550'):  # File not found
                                print(f"服务器上未找到文件 {obs_filename}，尝试其他后缀...")
                                break  # 跳出尝试这个后缀的循环，换下一个后缀
                            else:
                                raise
                        except error_temp as e:
                            if attempt < max_retries - 1:
                                print(f"临时错误: {e}。{retry_delay} 秒后重试...")
                                time.sleep(retry_delay)
                            else:
                                print(f"尝试 {max_retries} 次后仍无法下载 {obs_filename}。")
                                break
                        except Exception as e:
                            if 'EOF occurred in violation of protocol (_ssl.c:2427)' in str(e):
                                print('发生EOF错误，正在重新连接FTP服务器...')
                                ftp = connect_to_ftp()
                            else:
                                print(f'处理 {local_obs_gz_filename} 时出错: {e}')
                                break

                    if file_downloaded:
                        break  # 如果成功下载了文件，跳出尝试其他后缀的循环

                if not file_downloaded:
                    print(f"跳过 {single_date.strftime('%Y-%m-%d')} 的文件下载。")

        # 下载sp3文件
        if download_sp3:
            sp3_filename = f'COD0MGXFIN_{year}{doy_padded}0000_01D_05M_ORB.SP3.gz'
            sp3_directory = f'/pub/gps/products/{gps_week}/'
            local_sp3_gz_filename = os.path.join(save_path_sp3, sp3_filename)
            local_sp3_filename = os.path.join(save_path_sp3, f'COD0MGXFIN_{year}{doy_padded}0000_01D_05M_ORB.SP3')

            # 检查文件是否已经存在
            if os.path.exists(local_sp3_filename):
                print(f'{local_sp3_filename} 已存在，跳过下载。')
            else:
                max_retries = 3
                retry_delay = 5
                for attempt in range(max_retries):
                    try:
                        print(f"正在下载 {sp3_filename}...")
                        with open(local_sp3_gz_filename, 'wb') as file:
                            ftp.retrbinary(f'RETR {sp3_directory}{sp3_filename}', file.write)
                        print(f'成功下载 {local_sp3_gz_filename}')

                        print(f"正在解压 {local_sp3_gz_filename}...")
                        with gzip.open(local_sp3_gz_filename, 'rb') as f_in:
                            with open(local_sp3_filename, 'wb') as f_out:
                                shutil.copyfileobj(f_in, f_out)
                        print(f'成功解压到 {local_sp3_filename}')

                        # 删除压缩文件
                        os.remove(local_sp3_gz_filename)
                        print(f"删除压缩文件 {local_sp3_gz_filename}")
                        break
                    except error_perm as e:
                        if str(e).startswith('550'):  # File not found
                            print(f"服务器上未找到文件 {sp3_filename}，跳过。")
                            break
                        else:
                            raise
                    except error_temp as e:
                        if attempt < max_retries - 1:
                            print(f"临时错误: {e}。{retry_delay} 秒后重试...")
                            time.sleep(retry_delay)
                        else:
                            print(f"尝试 {max_retries} 次后仍无法下载 {sp3_filename}。")
                            break
                    except Exception as e:
                        if 'EOF occurred in violation of protocol (_ssl.c:2427)' in str(e):
                            print('发生EOF错误，正在重新连接FTP服务器...')
                            ftp = connect_to_ftp()
                        else:
                            print(f'处理 {local_sp3_gz_filename} 时出错: {e}')
                            break

        day_count += 1

        # 每下载10天的文件输出一次计时信息
        if day_count % 10 == 0:
            elapsed_time = time.time() - start_time
            print(f'已下载 {day_count} 天的文件，耗时: {elapsed_time:.2f} 秒')

    # 断开FTP连接
    print("正在关闭FTP连接...")
    ftp.quit()
    print("FTP连接已成功关闭。")

In [46]:
# 示例使用
if __name__ == "__main__":
    save_path_obs = f"/content/data/obs/{station_name}/{start_date.year}"
    save_path_sp3 = f'/content/data/sp3/{start_date.year}'

    print("开始下载和提取文件...")
    download_and_extract_files(start_date, end_date, save_path_obs, save_path_sp3, station_name, obs_suffix, download_obs=True, download_sp3=True)
    print("下载和提取过程完成。")

流式输出内容被截断，只能显示最后 5000 行内容。
成功下载 /content/data/sp3/2020/COD0MGXFIN_20203210000_01D_05M_ORB.SP3.gz
正在解压 /content/data/sp3/2020/COD0MGXFIN_20203210000_01D_05M_ORB.SP3.gz...
成功解压到 /content/data/sp3/2020/COD0MGXFIN_20203210000_01D_05M_ORB.SP3
删除压缩文件 /content/data/sp3/2020/COD0MGXFIN_20203210000_01D_05M_ORB.SP3.gz

正在处理日期: 2020-11-17
正在下载 SAVO00BRA_R_20203220000_01D_30S_MO.crx.gz...
成功下载 /content/data/obs/SAVO/2020/SAVO00BRA_R_20203220000_01D_30S_MO.crx.gz
正在解压 /content/data/obs/SAVO/2020/SAVO00BRA_R_20203220000_01D_30S_MO.crx.gz...
成功解压到 /content/data/obs/SAVO/2020/SAVO00BRA_R_20203220000_01D_30S_MO.crx
删除压缩文件 /content/data/obs/SAVO/2020/SAVO00BRA_R_20203220000_01D_30S_MO.crx.gz
正在下载 COD0MGXFIN_20203220000_01D_05M_ORB.SP3.gz...
成功下载 /content/data/sp3/2020/COD0MGXFIN_20203220000_01D_05M_ORB.SP3.gz
正在解压 /content/data/sp3/2020/COD0MGXFIN_20203220000_01D_05M_ORB.SP3.gz...
成功解压到 /content/data/sp3/2020/COD0MGXFIN_20203220000_01D_05M_ORB.SP3
删除压缩文件 /content/data/sp3/2020/COD0MGXFIN_20203220000_01D

检查文件夹/content/data/obs/下的所有子文件（包括子文件夹中的文件）是否有.gz结尾的，如果有，删除这些文件

In [47]:
import os

# 指定文件夹路径
folder_path = '/content/data/obs/'

# 存储被删除的文件
deleted_files = []

# 遍历文件夹及其子文件夹
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.gz'):
            # 构造完整文件路径
            file_path = os.path.join(root, file)
            # 删除文件
            os.remove(file_path)
            # 记录被删除的文件
            deleted_files.append(file_path)

# 打印删除的文件数量和名称
print(f'删除的文件数量: {len(deleted_files)}')
print('删除的文件:')
for deleted_file in deleted_files:
    print(deleted_file)


删除的文件数量: 46
删除的文件:
/content/data/obs/SAVO/2020/SAVO00BRA_S_20200130000_01D_30S_MO.crx.gz
/content/data/obs/SAVO/2020/SAVO00BRA_R_20200490000_01D_30S_MO.crx.gz
/content/data/obs/SAVO/2020/SAVO00BRA_R_20201890000_01D_30S_MO.crx.gz
/content/data/obs/SAVO/2020/SAVO00BRA_S_20203370000_01D_30S_MO.crx.gz
/content/data/obs/SAVO/2020/SAVO00BRA_S_20203170000_01D_30S_MO.crx.gz
/content/data/obs/SAVO/2020/SAVO00BRA_S_20201890000_01D_30S_MO.crx.gz
/content/data/obs/SAVO/2020/SAVO00BRA_R_20203190000_01D_30S_MO.crx.gz
/content/data/obs/SAVO/2020/SAVO00BRA_S_20203190000_01D_30S_MO.crx.gz
/content/data/obs/SAVO/2020/SAVO00BRA_R_20200130000_01D_30S_MO.crx.gz
/content/data/obs/SAVO/2020/SAVO00BRA_S_20201870000_01D_30S_MO.crx.gz
/content/data/obs/SAVO/2020/SAVO00BRA_S_20201900000_01D_30S_MO.crx.gz
/content/data/obs/SAVO/2020/SAVO00BRA_R_20201900000_01D_30S_MO.crx.gz
/content/data/obs/SAVO/2020/SAVO00BRA_S_20203080000_01D_30S_MO.crx.gz
/content/data/obs/SAVO/2020/SAVO00BRA_R_20203080000_01D_30S_MO.crx.gz
/

/content/data/obs/下的所有子文件（包括子文件夹中的文件）.crx，全部使用crx2rnx.exe进行转换

下载crx2rnx工具，并解压

In [48]:
import os

# 定义目标文件夹路径
target_dir = '/content/tool'

# 创建目标文件夹（如果不存在）
os.makedirs(target_dir, exist_ok=True)

# 定义下载链接和目标文件路径
download_url = 'https://terras.gsi.go.jp/ja/crx2rnx/RNXCMP_4.1.0_Linux_x86_32bit.tar.gz'
tar_file_path = os.path.join(target_dir, 'RNXCMP_4.1.0_Linux_x86_32bit.tar.gz')

# 使用 wget 下载文件
!wget -P {target_dir} {download_url}

# 解压缩文件
!tar -xzf {tar_file_path} -C {target_dir}

# 删除 .tar.gz 文件
os.remove(tar_file_path)

print(f"文件已下载并解压到 {target_dir} 并删除了 {tar_file_path}")

--2024-10-09 01:39:03--  https://terras.gsi.go.jp/ja/crx2rnx/RNXCMP_4.1.0_Linux_x86_32bit.tar.gz
Resolving terras.gsi.go.jp (terras.gsi.go.jp)... 163.42.5.1
Connecting to terras.gsi.go.jp (terras.gsi.go.jp)|163.42.5.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 497324 (486K) [application/x-gzip]
Saving to: ‘/content/tool/RNXCMP_4.1.0_Linux_x86_32bit.tar.gz’

RNXCMP_4.1.0_Linux_ 100%[===================>] 485.67K   588KB/s    in 0.8s    

2024-10-09 01:39:04 (588 KB/s) - ‘/content/tool/RNXCMP_4.1.0_Linux_x86_32bit.tar.gz’ saved [497324/497324]

文件已下载并解压到 /content/tool 并删除了 /content/tool/RNXCMP_4.1.0_Linux_x86_32bit.tar.gz


对所有data/obs下的所有子文件夹下的所有子文件，调用"/content/tool/RNXCMP_4.1.0_Linux_x86_32bit/bin/CRX2RNX {filename}"

此步耗时8min/2year。

In [49]:
import os
import subprocess

# 定义工具路径和数据目录路径
tool_path = '/content/tool/RNXCMP_4.1.0_Linux_x86_32bit/bin/CRX2RNX'
data_dir = '/content/data/obs'

# 遍历目录及其子目录
for root, dirs, files in os.walk(data_dir):
    for file in files:
        # 检查文件是否以 .crx 结尾
        if file.endswith('.crx'):
            # 构建文件路径
            file_path = os.path.join(root, file)
            rnx_file_path = os.path.join(root, file[:-4] + '.rnx')

            # 检查是否已经存在对应的 .rnx 文件
            if os.path.exists(rnx_file_path):
                print(f"Skipped: {file_path} (already converted to {rnx_file_path})")
                continue  # 跳过转换

            try:
                # 调用 CRX2RNX 工具
                command = f'{tool_path} {file_path}'
                subprocess.run(command, shell=True, check=True)
                print(f"Processed: {file_path}")
            except subprocess.CalledProcessError as e:
                print(f"Error processing {file_path}: {e}")
                continue  # 跳过错误并继续处理下一个文件

# 删除所有 .crx 文件
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.crx'):
            file_path = os.path.join(root, file)
            os.remove(file_path)
            print(f"Deleted: {file_path}")

Processed: /content/data/obs/SAVO/2020/SAVO00BRA_R_20203530000_01D_30S_MO.crx
Processed: /content/data/obs/SAVO/2020/SAVO00BRA_R_20200970000_01D_30S_MO.crx
Processed: /content/data/obs/SAVO/2020/SAVO00BRA_R_20201440000_01D_30S_MO.crx
Processed: /content/data/obs/SAVO/2020/SAVO00BRA_R_20202210000_01D_30S_MO.crx
Processed: /content/data/obs/SAVO/2020/SAVO00BRA_R_20202090000_01D_30S_MO.crx
Processed: /content/data/obs/SAVO/2020/SAVO00BRA_R_20203210000_01D_30S_MO.crx
Processed: /content/data/obs/SAVO/2020/SAVO00BRA_R_20200050000_01D_30S_MO.crx
Processed: /content/data/obs/SAVO/2020/SAVO00BRA_R_20201680000_01D_30S_MO.crx
Processed: /content/data/obs/SAVO/2020/SAVO00BRA_R_20200920000_01D_30S_MO.crx
Processed: /content/data/obs/SAVO/2020/SAVO00BRA_R_20202810000_01D_30S_MO.crx
Processed: /content/data/obs/SAVO/2020/SAVO00BRA_R_20201530000_01D_30S_MO.crx
Processed: /content/data/obs/SAVO/2020/SAVO00BRA_R_20200090000_01D_30S_MO.crx
Processed: /content/data/obs/SAVO/2020/SAVO00BRA_R_20202170000_0

# 2 处理得到载噪比数据

安装python需要的包

In [50]:
!pip install pyunpack patool

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 1.9 MB/s eta 0:00:00


下载项目，实现调用dataprocessing的代码

In [51]:
!git clone https://github.com/BlackiePiggy/SFPD.git

Cloning into 'SFPD'...
remote: Enumerating objects: 171, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 171 (delta 46), reused 156 (delta 31), pack-reused 0 (from 0)
Receiving objects: 100% (171/171), 1.02 MiB | 10.35 MiB/s, done.
Resolving deltas: 100% (46/46), done.


执行载噪比处理程序

读取obs文件耗时大概为200天/11minutes

总耗时约1hour/1year

In [ ]:
import sys
import os
# 将 /content/dataproc 文件夹添加到 Python 路径
sys.path.append('/content/SFPD/dataprocessing')
sys.path.append('/content/SFPD/dataprocessing/lib')

import read_CN_from_obs as readCN
import read_el_from_sp3 as readEL
import combine_cn_el as combineCNEL
import elFilter as elft

################################################################################################
def generate_satellite_range(prefix, start, end):
    return [f'{prefix}{str(i).zfill(2)}' for i in range(start, end + 1)]

satellite_range = generate_satellite_range('G', 1, 32)  # 生成要处理的卫星范围，可自选

for year in range(start_year, end_year+1):  # 遍历2022到2023年
    test_start_date = f'{year}001'
    test_end_date = f'{year}365'

    #########################################文件路径################################################
    test_input_obs_folder = f'{base_data_path}/obs/{station_name}/{year}'  # 替换为你的输入数据目录路径
    test_input_sp3_dir = f'{base_data_path}/sp3/{year}'
    test_output_cn_folder = f"{base_data_path}/1_time_CN/{year}"  # 替换为你的输出数据目录路径
    test_output_el_folder = f'{base_data_path}/2_time_EL/{year}'
    test_output_cn_el_folder = f'{base_data_path}/3_time_CN_EL/{year}'
    test_time_cn_el_filtered_folder = f'{base_data_path}/4_filtered_time_CN/{year}'
    ######################################################################

    # 第一步：读取所有obs文件，提取CN值，存储到timeCN文件夹中
    readCN.read_CN_value_from_obs_AAO(station_name, SS_variables, 'all', test_input_obs_folder, test_output_cn_folder, test_start_date, test_end_date)  # 正常日期

    readEL.read_el_from_sp3(station_name, test_input_sp3_dir, test_output_el_folder, satellite_range, station_llh, test_start_date, test_end_date)

    combineCNEL.merge_cn_el_files(station_name, SS_variables, satellite_range, test_output_cn_folder, test_output_el_folder, test_output_cn_el_folder, test_start_date, test_end_date)  # 正常日期

    elft.filter_cn_el_files(station_name, SS_variables, satellite_range, test_output_cn_el_folder, test_time_cn_el_filtered_folder, test_start_date, test_end_date, el_mask)  # 正常日期

    print(f"Test Dataset for year {year} Generated Successfully!")
    ######################################################################


Created directory: /content/data/1_time_CN/2020
/content/data/obs/SAVO/2020/SAVO00BRA_R_20200010000_01D_30S_MO.rnx exist in working directory | Reading...
Observation file  /content/data/obs/SAVO/2020/SAVO00BRA_R_20200010000_01D_30S_MO.rnx  is read in 4.85 seconds.
Saved: /content/data/1_time_CN/2020/SAVO_2020001_CN_G01_S2W.csv
Saved: /content/data/1_time_CN/2020/SAVO_2020001_CN_G03_S2W.csv
Saved: /content/data/1_time_CN/2020/SAVO_2020001_CN_G04_S2W.csv
Saved: /content/data/1_time_CN/2020/SAVO_2020001_CN_G08_S2W.csv
Saved: /content/data/1_time_CN/2020/SAVO_2020001_CN_G09_S2W.csv
Saved: /content/data/1_time_CN/2020/SAVO_2020001_CN_G10_S2W.csv
Saved: /content/data/1_time_CN/2020/SAVO_2020001_CN_G11_S2W.csv
Saved: /content/data/1_time_CN/2020/SAVO_2020001_CN_G14_S2W.csv
Saved: /content/data/1_time_CN/2020/SAVO_2020001_CN_G16_S2W.csv
Saved: /content/data/1_time_CN/2020/SAVO_2020001_CN_G22_S2W.csv
Saved: /content/data/1_time_CN/2020/SAVO_2020001_CN_G23_S2W.csv
Saved: /content/data/1_time_CN

生成数据集，最终生成数据集存放于data/dataset文件夹中

运行时间：1min/1year

In [ ]:
import os
import pandas as pd


# 遍历起始年份和结束年份之间的每一年
for year in range(start_year, end_year + 1):
    year = str(year)  # 转换年份为字符串
    date_start = f'{year}001'
    date_end = f'{year}365'

    # 定义输入输出路径
    test_data_path = rf'/content/data/4_filtered_time_CN/{year}'
    output_base_path = rf'/content/drive/MyDrive/Colab Notebooks/SFPD/dataset/{year}/{station_name}'

    for satellite in satellites:
        # 动态设置输出路径
        output_path = os.path.join(output_base_path, f'{station_name}_{year}_{signal}_{satellite}.csv')

        # 创建一个空的列表来存储数据框
        data_frames = []

        # 遍历所有文件
        for path in [test_data_path]:
            for filename in os.listdir(path):
                if filename.startswith(station_name) and filename.endswith('_filtered.csv'):
                    parts = filename.split('_')
                    date_str = parts[1]
                    satellite_str = parts[3]
                    signal_str = parts[4]

                    # 确保文件的日期在范围内，并匹配卫星号和信号
                    if date_start <= date_str <= date_end and satellite_str == satellite and signal_str == signal:
                        file_path = os.path.join(path, filename)
                        df = pd.read_csv(file_path)
                        data_frames.append(df)

        # 如果有符合条件的数据框则进行处理
        if data_frames:
            # 合并所有数据框
            merged_df = pd.concat(data_frames, ignore_index=True)

            # 删除第三列
            merged_df.drop(merged_df.columns[2], axis=1, inplace=True)

            # 确认Timestamp列存在并进行格式转换
            if 'Timestamp' in merged_df.columns:
                merged_df['Timestamp'] = pd.to_datetime(merged_df['Timestamp'])

            # 按日期排序
            merged_df.sort_values(by='Timestamp', inplace=True)

            # 输出合并后的数据框到指定文件
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            merged_df.to_csv(output_path, index=False)

            print(f"合并后的文件已保存到: {output_path}")
        else:
            print(f"没有找到符合条件的文件：年份 {year} 卫星 {satellite}")


# 3 SFPD检测算法

安装必要的python包

In [ ]:
!pip install fastdtw

执行SFPD算法

运行速度：

In [ ]:
# main.py
import sys
import os
# 将 /content/dataproc 文件夹添加到 Python 路径
sys.path.append('/content/SFPD')

from sfpd import calculate_sfpd

timestamp_format = '%Y-%m-%d %H:%M:%S'
residual_threshold = 5
save_plots = True  # 控制是否保存每日图像的布尔变量
figsize = (15, 5)  # 图形的大小 (宽, 高)
plot_dpi = 100  # 图形的分辨率
input_data_path = '/content/drive/MyDrive/Colab Notebooks/SFPD/dataset'  # 输入CSV文件所在目录

# 自定义输出路径
output_image_dir_base = '/content/drive/MyDrive/Colab Notebooks/SFPD/DTW_plot_result'  # 图像输出基础路径
output_DTW_dir_base = '/content/drive/MyDrive/Colab Notebooks/SFPD/DTW_result'  # DTW输出路径

# 调用SFPD计算函数
calculate_sfpd(years, stations, SS_variables, svn_range, timestamp_format, residual_threshold, save_plots, figsize, plot_dpi,
               input_data_path, output_image_dir_base, output_DTW_dir_base)


# 4 异常值判断

## 4.1 离群值检测



### 方法一 IQR法

此处暂时考虑采用IQR的方法对异常值进行判断。

先将DTW的结果放置于/content/drive/MyDrive/Colab Notebooks/SFPD/DTW_result中，选择正确的station, year, signal后，运行下面代码。

经测试，multiplier = 9时，具有比较好的分离效果。

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from collections import defaultdict

# Step 0: Initialize
station = 'CAS1'  # Replace with actual station name
year = '2021'  # Replace with actual year
signal = 'S2W'  # Replace with actual signal name

# Set input and output folders
input_folder = r'/content/drive/MyDrive/Colab Notebooks/DTW_result'
output_folder = r'/content/drive/MyDrive/Colab Notebooks/DTW_IQR_split_result'
consolidated_output_folder = r'/content/drive/MyDrive/Colab Notebooks/data'
image_output_folder = r'/content/drive/MyDrive/Colab Notebooks/outlier_detection_plots'

# Create output folders if they don't exist
for folder in [output_folder, consolidated_output_folder, image_output_folder]:
    os.makedirs(folder, exist_ok=True)

# Initialize variables to store all dates and data
all_dates = set()
all_data = defaultdict(dict)


# Define date to DOY function
def date_to_doy(date):
    return date.timetuple().tm_yday


# Step 1: Process each satellite
for sat_num in range(1, 33):
    sat = f'G{sat_num:02d}'  # Generate satellite ID (G01, G02, ..., G32)

    # Construct file path
    file_path = os.path.join(input_folder, f'{station}_{year}_{signal}_{sat}_DTW.csv')

    # Check if the file exists before attempting to read it
    if os.path.isfile(file_path):
        # Read data, handling non-numeric elements as NaN
        data = pd.read_csv(file_path, parse_dates=[0])
        dates = data.iloc[:, 0]
        values = pd.to_numeric(data.iloc[:, 1], errors='coerce')

        # Method 1: IQR (Interquartile Range) method
        multiplier = 9  # Default is 1.5, can be manually adjusted

        Q1 = values.quantile(0.25)
        Q3 = values.quantile(0.75)
        IQR = Q3 - Q1

        # Adjust upper bound based on multiplier
        upper_bound = Q3 + multiplier * IQR

        # Mark outliers
        is_outlier = values > upper_bound

        # Record outlier dates and values
        outlier_dates = dates[is_outlier]
        outlier_values = values[is_outlier]

        # Plot and save figure
        plt.figure(figsize=(12, 6))
        plt.scatter(dates[~is_outlier], values[~is_outlier], c='b', label='Non-Outlier Data')
        plt.scatter(outlier_dates, outlier_values, c='r', label='Outliers')
        plt.axhline(y=upper_bound, color='r', linestyle='--', label=f'Upper Threshold: {upper_bound:.2f}')

        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.title(f'Time Series Data with Outliers - {station} {year} - {sat}')
        plt.legend()

        image_path = os.path.join(image_output_folder, f'{station}_{year}_{signal}_{sat}_OutlierDetection.png')
        plt.savefig(image_path)
        plt.close()

        # Calculate DOY
        doy = dates.apply(date_to_doy)

        # Output results as CSV file
        output_df = pd.DataFrame({'Date': dates, 'DOY': doy, 'IsOutlier': is_outlier.astype(int)})
        output_path = os.path.join(output_folder, f'{station}_{year}_{signal}_{sat}_OutlierDetectionResults.csv')
        output_df.to_csv(output_path, index=False)

        # Add dates to all_dates
        all_dates.update(dates)

        # Store satellite data
        all_data[sat_num] = dict(zip(dates.astype(str), is_outlier.astype(int)))
    else:
        print(f'File not found: {file_path}')

# Step 2: Consolidate data
all_dates = sorted(all_dates)
all_doy = [date_to_doy(date) for date in all_dates]

# Create output data DataFrame
output_data = pd.DataFrame({'Date': all_dates, 'DOY': all_doy})

# Fill data
for sat_num in range(1, 33):
    sat = f'G{sat_num:02d}'
    if sat_num in all_data:
        output_data[sat] = output_data['Date'].astype(str).map(all_data[sat_num])

# Write to CSV file
consolidated_output_path = os.path.join(consolidated_output_folder,
                                        f'{station}_{year}_{signal}_ConsolidatedResults.csv')
output_data.to_csv(consolidated_output_path, index=False)

print('Processing completed.')
print(f'Outlier detection results saved to: {output_folder}')
print(f'Outlier detection plots saved to: {image_output_folder}')
print(f'Consolidated file saved to: {consolidated_output_path}')

### 方法二 k-means聚类

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Step 0: Initialize
station = 'BAIE'  # Replace with actual station name
year = '2020'  # Replace with actual year
signal = 'S2W'  # Replace with actual signal name

# Set input and output folders
input_folder = r'/content/drive/MyDrive/Colab Notebooks/DTW_result'
output_folder = r'/content/drive/MyDrive/Colab Notebooks/DTW_kmeans_split_result'
consolidated_output_folder = r'/content/drive/MyDrive/Colab Notebooks/data_kmeans'
image_output_folder = r'/content/drive/MyDrive/Colab Notebooks/kmeans_outlier_detection_plots'

# Create output folders if they don't exist
for folder in [output_folder, consolidated_output_folder, image_output_folder]:
    os.makedirs(folder, exist_ok=True)

# Initialize variables to store all dates and data
all_dates = set()
all_data = defaultdict(dict)

# Define date to DOY function
def date_to_doy(date):
    return date.timetuple().tm_yday

# Step 1: Process each satellite
for sat_num in range(1, 33):
    sat = f'G{sat_num:02d}'  # Generate satellite ID (G01, G02, ..., G32)

    # Construct file path
    file_path = os.path.join(input_folder, f'{station}_{year}_{signal}_{sat}_DTW.csv')

    # Check if the file exists before attempting to read it
    if os.path.isfile(file_path):
        # Read data, handling non-numeric elements as NaN
        data = pd.read_csv(file_path, parse_dates=[0])
        dates = data.iloc[:, 0]
        values = pd.to_numeric(data.iloc[:, 1], errors='coerce')

        # Remove NaN values
        valid_mask = ~values.isna()
        valid_dates = dates[valid_mask]
        valid_values = values[valid_mask]

        if len(valid_values) > 1:  # Ensure we have at least two valid data points for clustering
            # Prepare data for k-means
            X = np.array(valid_values).reshape(-1, 1)
            X = StandardScaler().fit_transform(X)

            # Apply k-means clustering
            kmeans = KMeans(n_clusters=2, random_state=42)
            labels = kmeans.fit_predict(X)

            # Determine which cluster contains outliers (assume it's the smaller cluster)
            cluster_sizes = np.bincount(labels)
            outlier_cluster = np.argmin(cluster_sizes)

            # Mark outliers
            is_outlier = labels == outlier_cluster

            # Create a Series with all dates, marking NaN dates and non-outliers as False
            all_is_outlier = pd.Series(False, index=dates)
            all_is_outlier[valid_dates[is_outlier]] = True

            # Plot and save figure
            plt.figure(figsize=(12, 6))
            plt.scatter(valid_dates[~is_outlier], valid_values[~is_outlier], c='b', label='Non-Outlier Data')
            plt.scatter(valid_dates[is_outlier], valid_values[is_outlier], c='r', label='Outliers')
            plt.scatter(dates[~valid_mask], values[~valid_mask], c='gray', alpha=0.5, label='NaN Data')

            plt.xlabel('Date')
            plt.ylabel('Value')
            plt.title(f'Time Series Data with Outliers (K-means) - {station} {year} - {sat}')
            plt.legend()

            image_path = os.path.join(image_output_folder, f'{station}_{year}_{signal}_{sat}_KmeansOutlierDetection.png')
            plt.savefig(image_path)
            plt.close()

            # Calculate DOY
            doy = dates.apply(date_to_doy)

            # Output results as CSV file
            output_df = pd.DataFrame({'Date': dates, 'DOY': doy, 'IsOutlier': all_is_outlier.astype(int)})
            output_path = os.path.join(output_folder, f'{station}_{year}_{signal}_{sat}_KmeansOutlierDetectionResults.csv')
            output_df.to_csv(output_path, index=False)

            # Add dates to all_dates
            all_dates.update(dates)

            # Store satellite data
            all_data[sat_num] = dict(zip(dates.astype(str), all_is_outlier.astype(int)))
        else:
            print(f'Not enough valid data points for clustering: {file_path}')
    else:
        print(f'File not found: {file_path}')

# Step 2: Consolidate data
all_dates = sorted(all_dates)
all_doy = [date_to_doy(date) for date in all_dates]

# Create output data DataFrame
output_data = pd.DataFrame({'Date': all_dates, 'DOY': all_doy})

# Fill data
for sat_num in range(1, 33):
    sat = f'G{sat_num:02d}'
    if sat_num in all_data:
        output_data[sat] = output_data['Date'].astype(str).map(all_data[sat_num])

# Write to CSV file
consolidated_output_path = os.path.join(consolidated_output_folder,
                                        f'{station}_{year}_{signal}_KmeansConsolidatedResults.csv')
output_data.to_csv(consolidated_output_path, index=False)

print('Processing completed.')
print(f'K-means outlier detection results saved to: {output_folder}')
print(f'K-means outlier detection plots saved to: {image_output_folder}')
print(f'Consolidated file saved to: {consolidated_output_path}')

### 方法三 SVM

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM

# Step 0: Initialize
station = 'BAIE'  # Replace with actual station name
year = '2021'  # Replace with actual year
signal = 'S2W'  # Replace with actual signal name

# Set input and output folders
input_folder = r'/content/drive/MyDrive/Colab Notebooks/DTW_result'
output_folder = r'/content/drive/MyDrive/Colab Notebooks/DTW_SVM_split_result'
consolidated_output_folder = r'/content/drive/MyDrive/Colab Notebooks/data_SVM'
image_output_folder = r'/content/drive/MyDrive/Colab Notebooks/SVM_outlier_detection_plots'

# Create output folders if they don't exist
for folder in [output_folder, consolidated_output_folder, image_output_folder]:
    os.makedirs(folder, exist_ok=True)

# Initialize variables to store all dates and data
all_dates = set()
all_data = defaultdict(dict)

# Define date to DOY function
def date_to_doy(date):
    return date.timetuple().tm_yday

# Step 1: Process each satellite
for sat_num in range(1, 33):
    sat = f'G{sat_num:02d}'  # Generate satellite ID (G01, G02, ..., G32)

    # Construct file path
    file_path = os.path.join(input_folder, f'{station}_{year}_{signal}_{sat}_DTW.csv')

    # Check if the file exists before attempting to read it
    if os.path.isfile(file_path):
        # Read data, handling non-numeric elements as NaN
        data = pd.read_csv(file_path, parse_dates=[0])
        dates = data.iloc[:, 0]
        values = pd.to_numeric(data.iloc[:, 1], errors='coerce')

        # Remove NaN values
        valid_mask = ~values.isna()
        valid_dates = dates[valid_mask]
        valid_values = values[valid_mask]

        if len(valid_values) > 1:  # Ensure we have at least two valid data points for SVM
            # Prepare data for One-Class SVM
            X = np.array(valid_values).reshape(-1, 1)
            X = StandardScaler().fit_transform(X)

            # Apply One-Class SVM
            svm = OneClassSVM(kernel='rbf', nu=0.1)  # nu is the upper bound on the fraction of outliers
            is_inlier = svm.fit_predict(X)
            is_outlier = is_inlier == -1  # One-Class SVM marks outliers as -1

            # Create a Series with all dates, marking NaN dates and non-outliers as False
            all_is_outlier = pd.Series(False, index=dates)
            all_is_outlier[valid_dates[is_outlier]] = True

            # Plot and save figure
            plt.figure(figsize=(12, 6))
            plt.scatter(valid_dates[~is_outlier], valid_values[~is_outlier], c='b', label='Non-Outlier Data')
            plt.scatter(valid_dates[is_outlier], valid_values[is_outlier], c='r', label='Outliers')
            plt.scatter(dates[~valid_mask], values[~valid_mask], c='gray', alpha=0.5, label='NaN Data')

            plt.xlabel('Date')
            plt.ylabel('Value')
            plt.title(f'Time Series Data with Outliers (One-Class SVM) - {station} {year} - {sat}')
            plt.legend()

            image_path = os.path.join(image_output_folder, f'{station}_{year}_{signal}_{sat}_SVMOutlierDetection.png')
            plt.savefig(image_path)
            plt.close()

            # Calculate DOY
            doy = dates.apply(date_to_doy)

            # Output results as CSV file
            output_df = pd.DataFrame({'Date': dates, 'DOY': doy, 'IsOutlier': all_is_outlier.astype(int)})
            output_path = os.path.join(output_folder, f'{station}_{year}_{signal}_{sat}_SVMOutlierDetectionResults.csv')
            output_df.to_csv(output_path, index=False)

            # Add dates to all_dates
            all_dates.update(dates)

            # Store satellite data
            all_data[sat_num] = dict(zip(dates.astype(str), all_is_outlier.astype(int)))
        else:
            print(f'Not enough valid data points for SVM: {file_path}')
    else:
        print(f'File not found: {file_path}')

# Step 2: Consolidate data
all_dates = sorted(all_dates)
all_doy = [date_to_doy(date) for date in all_dates]

# Create output data DataFrame
output_data = pd.DataFrame({'Date': all_dates, 'DOY': all_doy})

# Fill data
for sat_num in range(1, 33):
    sat = f'G{sat_num:02d}'
    if sat_num in all_data:
        output_data[sat] = output_data['Date'].astype(str).map(all_data[sat_num])

# Write to CSV file
consolidated_output_path = os.path.join(consolidated_output_folder,
                                        f'{station}_{year}_{signal}_SVMConsolidatedResults.csv')
output_data.to_csv(consolidated_output_path, index=False)

print('Processing completed.')
print(f'SVM outlier detection results saved to: {output_folder}')
print(f'SVM outlier detection plots saved to: {image_output_folder}')
print(f'Consolidated file saved to: {consolidated_output_path}')

### 方法四 DBSCAN密度聚类方法

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

# Step 0: Initialize
station = 'BAIE'  # Replace with actual station name
year = '2020'  # Replace with actual year
signal = 'S2W'  # Replace with actual signal name

# Set input and output folders
input_folder = r'/content/drive/MyDrive/Colab Notebooks/DTW_result'
output_folder = r'/content/drive/MyDrive/Colab Notebooks/DTW_DBSCAN_split_result'
consolidated_output_folder = r'/content/drive/MyDrive/Colab Notebooks/data_DBSCAN'
image_output_folder = r'/content/drive/MyDrive/Colab Notebooks/DBSCAN_outlier_detection_plots'

# Create output folders if they don't exist
for folder in [output_folder, consolidated_output_folder, image_output_folder]:
    os.makedirs(folder, exist_ok=True)

# Initialize variables to store all dates and data
all_dates = set()
all_data = defaultdict(dict)

# Define date to DOY function
def date_to_doy(date):
    return date.timetuple().tm_yday

# Step 1: Process each satellite
for sat_num in range(1, 33):
    sat = f'G{sat_num:02d}'  # Generate satellite ID (G01, G02, ..., G32)

    # Construct file path
    file_path = os.path.join(input_folder, f'{station}_{year}_{signal}_{sat}_DTW.csv')

    # Check if the file exists before attempting to read it
    if os.path.isfile(file_path):
        # Read data, handling non-numeric elements as NaN
        data = pd.read_csv(file_path, parse_dates=[0])
        dates = data.iloc[:, 0]
        values = pd.to_numeric(data.iloc[:, 1], errors='coerce')

        # Remove NaN values
        valid_mask = ~values.isna()
        valid_dates = dates[valid_mask]
        valid_values = values[valid_mask]

        if len(valid_values) > 1:  # Ensure we have at least two valid data points for DBSCAN
            # Prepare data for DBSCAN
            X = np.array(valid_values).reshape(-1, 1)
            X = StandardScaler().fit_transform(X)

            # Apply DBSCAN
            dbscan = DBSCAN(eps=0.3, min_samples=7)
            labels = dbscan.fit_predict(X)

            # In DBSCAN, outliers are labeled as -1
            is_outlier = labels == -1

            # Create a Series with all dates, marking NaN dates and non-outliers as False
            all_is_outlier = pd.Series(False, index=dates)
            all_is_outlier[valid_dates[is_outlier]] = True

            # Plot and save figure
            plt.figure(figsize=(12, 6))
            plt.scatter(valid_dates[~is_outlier], valid_values[~is_outlier], c='b', label='Non-Outlier Data')
            plt.scatter(valid_dates[is_outlier], valid_values[is_outlier], c='r', label='Outliers')
            plt.scatter(dates[~valid_mask], values[~valid_mask], c='gray', alpha=0.5, label='NaN Data')

            plt.xlabel('Date')
            plt.ylabel('Value')
            plt.title(f'Time Series Data with Outliers (DBSCAN) - {station} {year} - {sat}')
            plt.legend()

            image_path = os.path.join(image_output_folder, f'{station}_{year}_{signal}_{sat}_DBSCANOutlierDetection.png')
            plt.savefig(image_path)
            plt.close()

            # Calculate DOY
            doy = dates.apply(date_to_doy)

            # Output results as CSV file
            output_df = pd.DataFrame({'Date': dates, 'DOY': doy, 'IsOutlier': all_is_outlier.astype(int)})
            output_path = os.path.join(output_folder, f'{station}_{year}_{signal}_{sat}_DBSCANOutlierDetectionResults.csv')
            output_df.to_csv(output_path, index=False)

            # Add dates to all_dates
            all_dates.update(dates)

            # Store satellite data
            all_data[sat_num] = dict(zip(dates.astype(str), all_is_outlier.astype(int)))
        else:
            print(f'Not enough valid data points for DBSCAN: {file_path}')
    else:
        print(f'File not found: {file_path}')

# Step 2: Consolidate data
all_dates = sorted(all_dates)
all_doy = [date_to_doy(date) for date in all_dates]

# Create output data DataFrame
output_data = pd.DataFrame({'Date': all_dates, 'DOY': all_doy})

# Fill data
for sat_num in range(1, 33):
    sat = f'G{sat_num:02d}'
    if sat_num in all_data:
        output_data[sat] = output_data['Date'].astype(str).map(all_data[sat_num])

# Write to CSV file
consolidated_output_path = os.path.join(consolidated_output_folder,
                                        f'{station}_{year}_{signal}_DBSCANConsolidatedResults.csv')
output_data.to_csv(consolidated_output_path, index=False)

print('Processing completed.')
print(f'DBSCAN outlier detection results saved to: {output_folder}')
print(f'DBSCAN outlier detection plots saved to: {image_output_folder}')
print(f'Consolidated file saved to: {consolidated_output_path}')

## 4.2 离群值检测结果合成

IQR结果放置在data文件夹下。
以 *BIK0_2021_S2W_ConsolidatedResults.csv* 为例，文件第一列是年月日，第二列是DOY，第3-34列是卫星号。

下面采用合成的策略是，当某一日至少有5颗卫星判断为异常，那么这一天被判定为异常。最后综合判断结果输出到单独的csv文件中，这个新的文件前两列不变，第三列为综合判断结果。

In [ ]:
import pandas as pd
import numpy as np
import os

# 定义基本名称和文件扩展名
base_name = 'CAS1'
year = '2020'
signal = 'S2W'
input_name = f'{base_name}_{year}_{signal}'
input_file_name = f'{input_name}_ConsolidatedResults.csv'
output_file_name = f'{input_name}_combine.csv'

input_file_path = f'/content/data/{input_file_name}'
output_file_path = f'/content/combine/{output_file_name}'

# 读取CSV文件
df = pd.read_csv(input_file_path)

# 检查并处理空值
df.fillna(0, inplace=True)

# 计算每一天的32颗卫星时间序列数据的和
df['Sum'] = df.iloc[:, 2:34].sum(axis=1)

# 判断每一天是否为异常日
df['Is_Abnormal'] = df['Sum'].apply(lambda x: 'Abnormal' if x >= 5 else 'normal')

# 删除中间计算的Sum列
df.drop(columns=['Sum'], inplace=True)

# 只保留前两列和Is_Abnormal列
result_df = df[['Date', 'DOY', 'Is_Abnormal']]

# 确保输出目录存在，如果不存在则创建
output_dir = os.path.dirname(output_file_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 保存结果到新的CSV文件
result_df.to_csv(output_file_path, index=False)

print(f"结果已保存到 {output_file_path}")


## 4.3 多站结果综合

对多个测站取并集，得到最终判断结果。

如果all_base = 0， 表示按照 base_names 进行选择；

如果all_base = 1， 表示选择input_dir下的所有测站。

In [ ]:
import pandas as pd
import os

# 定义测站名称、年份和信号
base_names = ['CAS1', 'BAIE', 'BIK0', 'AIRA']
year = '2021'
signal = 'S2W'
input_dir = '/content/combine/'
output_dir = '/content/result/'

# 设定是否选择所有测站的标志
all_base = 0  # 1 表示选择所有测站，0 表示按照 base_names 进行选择

# 初始化一个空的 DataFrame 用于合并结果
merged_df = pd.DataFrame()

# 如果 all_base 为 1，选择所有测站文件；如果为 0，选择指定的测站
if all_base == 1:
    # 遍历 input_dir 中的所有 CSV 文件
    for file_name in os.listdir(input_dir):
        if file_name.endswith('_combine.csv'):
            file_path = os.path.join(input_dir, file_name)
            df = pd.read_csv(file_path)

            # 确保列存在
            if 'Date' in df.columns and 'DOY' in df.columns and 'Is_Abnormal' in df.columns:
                # 确保日期格式一致
                df['Date'] = pd.to_datetime(df['Date'])
                df['Year'] = df['Date'].dt.year

                # 仅保留对应年份的数据
                df = df[df['Year'] == int(year)]

                # 合并数据，保留 DOY 列
                merged_df = pd.concat([merged_df, df[['Date', 'DOY', 'Is_Abnormal']]], ignore_index=True)

else:
    # 遍历指定的测站名称
    for base_name in base_names:
        file_path = os.path.join(input_dir, f'{base_name}_combine.csv')

        # 读取 CSV 文件
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)

            # 确保列存在
            if 'Date' in df.columns and 'DOY' in df.columns and 'Is_Abnormal' in df.columns:
                # 确保日期格式一致
                df['Date'] = pd.to_datetime(df['Date'])
                df['Year'] = df['Date'].dt.year

                # 仅保留对应年份的数据
                df = df[df['Year'] == int(year)]

                # 合并数据，保留 DOY 列
                merged_df = pd.concat([merged_df, df[['Date', 'DOY', 'Is_Abnormal']]], ignore_index=True)

# 检查是否成功合并数据
if not merged_df.empty:
    # 处理合并后的数据，取并集
    final_df = merged_df.groupby(['Date', 'DOY']).agg({'Is_Abnormal': lambda x: 'Abnormal' if 'Abnormal' in x.values else 'normal'}).reset_index()

    # 生成输出文件名
    output_file_path = os.path.join(output_dir, f'{year}_{signal}_result.csv')

    # 确保输出目录存在，如果不存在则创建
    os.makedirs(output_dir, exist_ok=True)

    # 保存结果到新的 CSV 文件
    final_df.to_csv(output_file_path, index=False)

    print(f"结果已保存到 {output_file_path}")
else:
    print("未找到合适的数据，合并结果为空。")


## 4.4 可视化

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# 定义年份和信号
year = '2021'
signal = 'S2W'
input_dir = '/content/result/'
output_file_name = f'{year}_{signal}_result.csv'
input_file_path = os.path.join(input_dir, output_file_name)

# 读取结果文件
df = pd.read_csv(input_file_path)

# 将日期列转换为 datetime 格式
df['Date'] = pd.to_datetime(df['Date'])

# 将 Is_Abnormal 转换为 0 或 1
df['Value'] = df['Is_Abnormal'].apply(lambda x: 1 if x == 'Abnormal' else 0)

# 绘制图形
plt.figure(figsize=(12, 6))
plt.plot(df['Date'], df['Value'], marker='o', linestyle='-', color='b', markersize=4)
plt.title(f'{year} {signal} Abnormality Status')
plt.xlabel('Date')
plt.ylabel('Status (0 = normal, 1 = Abnormal)')
plt.xticks(rotation=45)
plt.yticks([0, 1], ['normal', 'Abnormal'])
plt.grid()

# 保存图形
output_img_path = os.path.join(input_dir, f'{year}{signal}result_plot.png')
plt.savefig(output_img_path)
plt.tight_layout()
plt.show()

print(f"图形已保存到 {output_img_path}")
